## Using IPyvolume

In [1]:
import numpy as np
import ipyvolume as ipv
V = np.zeros((128,128,128)) # our 3d array
# outer box
V[30:-30,30:-30,30:-30] = 0.75
V[35:-35,35:-35,35:-35] = 0.0
# inner box
V[50:-50,50:-50,50:-50] = 0.25
V[55:-55,55:-55,55:-55] = 0.0
ipv.quickvolshow(V, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)

/usr/local/lib/python3.6/dist-packages/ipyvolume/serialize.py:81: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


In [2]:
import ipyvolume as ipv
import numpy as np
x, y, z = np.random.random((3, 10000))
ipv.quickscatter(x, y, z, size=1, marker="sphere")

## Persistence diagrams with Gudhi

Next let us observe the persistence diagrams for two proteins. Tutorials on how to do it with Gudhi are available [here](https://github.com/GUDHI/TDA-tutorial).

In [4]:
from Bio.PDB import *
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pdbl_1 = PDBList()
pdbl_1.retrieve_pdb_file('4XP1')

pdbl_2 = PDBList()
pdbl_2.retrieve_pdb_file('3I40')

parser_1 = MMCIFParser()
parser_2 = MMCIFParser()

structure_1 = parser_1.get_structure('4XP1', 'xp/4xp1.cif')
structure_2 = parser_2.get_structure('4XP1', 'i4/3i40.cif')

Structure exists: '/workspace/hodgelaplacians/examples/xp/4xp1.cif' 
Structure exists: '/workspace/hodgelaplacians/examples/i4/3i40.cif' 


In [ ]:
import nglview as nv

view_1 = nv.show_biopython(structure_1)
view_1.clear_representations()
#view as ball and stick (atom and bond)
view_1.add_ball_and_stick()

view_1

In [ ]:
points_1 = []
points_2 = []

for model in structure_1:
    for chain in model:
        for residue in chain:
            for atom in residue:
                points_1.append(tuple(atom.coord))
                
for model in structure_2:
    for chain in model:
        for residue in chain:
            for atom in residue:
                points_2.append(tuple(atom.coord))

points_1 = np.array(points_1)
points_2 = np.array(points_2)

print(f"There are {len(points_1)} atoms in the protein 1.")
print(f"There are {len(points_2)} atoms in the protein 2.")

In [ ]:
import gudhi as gd
import time
start_time = time.time()


epsilon = 10 # Connect two points if distance is smaller than epsilon

skeleton_1  = gd.RipsComplex(points = points_1, max_edge_length=epsilon )
rips_simplex_tree_1 = skeleton_1.create_simplex_tree(max_dimension=3) 
rips_skeleton_gudhi_1 = rips_simplex_tree_1.get_skeleton(3)

skeleton_2  = gd.RipsComplex(points = points_2, max_edge_length=epsilon )
rips_simplex_tree_2 = skeleton_2.create_simplex_tree(max_dimension=3) 
rips_skeleton_gudhi_2 = rips_simplex_tree_2.get_skeleton(3)

end_time = time.time()

print(f"Simplex tree constructed in {end_time-start_time} seconds")

In [ ]:
import numpy as np
from pylab import *
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
# import sklearn_tda
#skeleton  = gd.RipsComplex(points = data_A_sample,max_edge_length=0.6 )

Rips_simplex_tree_sample = skeleton_1.create_simplex_tree(max_dimension=2) 

BarCodes_Rips = Rips_simplex_tree_sample.persistence()

gd.plot_persistence_diagram(BarCodes_Rips);